# 2 Wavepackets with Fluctuations

In [5]:
import matplotlib.animation as animation;import matplotlib.pyplot as plt; from matplotlib import gridspec
from labellines import labelLines; from celluloid import Camera; import scipy.signal as scs 
import scipy.special as ssp; from operator import add; import numpy as np; import random

## Simulation Parameters

In [6]:
nLat = 256; nTime = 32; width0 = 1.; amplitude0 = 0; nWid = 80; nAmp = 25

alpha = 8.
nCols = 5
n_cross = 2
nu = 2.*10**(-3)
omega = 0.25 * 50. * 2. * np.sqrt(nu)
delt = np.sqrt(nu / 2.) * (1. + 0.5)
lamb = delt * np.sqrt(2. / nu)
rho = 200. * 2. * np.sqrt(nu) / 2.**3
lenLat = 0.5 * 50. / np.sqrt(2. * nu)
nyq = int(nLat/2)+1
spec = int(nyq/2)
hLat = int(nLat/2)
dx = lenLat/nLat
dk = 2.*np.pi/lenLat
dt = dx/alpha
outsize = 4*n_cross*nLat/nTime
dtout = dt*outsize

m2eff = 4. * nu * (- 1. + lamb**2); print('m2eff=',m2eff) 
sigma = 0.15/np.sqrt(m2eff); print('sigma=',sigma)
phi0 = np.pi/2; print('phi0=',phi0)

Sims = [i for i in range(1, 21)]; print('sim = ', Sims)
width = [width0 + wid * 0.05 / np.sqrt(m2eff) for wid in range(1, nWid+1)]; print('width = ', width)
amplitude = [amplitude0 + np.pi/200. * amp for amp in range(1, nAmp+1)]; print('amplitude = ', amplitude)
amplitude += [np.pi/8 + np.pi/200. * amp for amp in range(1, 80+1)]; print('amplitude = ', amplitude)
mean1 = int(nLat/4 + nLat/10.); mean2 = nLat-mean1; print('means = ', mean1, mean2)
tean1 = int(mean1*nTime/nLat); tean2 = int(mean2*nTime/nLat)
lag = [int(i) for i in range(1, 3)]; print('lag = ', lag)

m2eff= 0.009999999999999997
sigma= 1.5000000000000002
phi0= 1.5707963267948966
sim =  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
width =  [1.5, 2.0, 2.500000000000001, 3.0000000000000004, 3.5000000000000004, 4.000000000000002, 4.500000000000002, 5.000000000000001, 5.500000000000001, 6.000000000000001, 6.500000000000002, 7.000000000000003, 7.500000000000002, 8.000000000000004, 8.500000000000002, 9.000000000000002, 9.500000000000004, 10.000000000000002, 10.500000000000004, 11.000000000000002, 11.500000000000004, 12.000000000000004, 12.500000000000004, 13.000000000000005, 13.500000000000004, 14.000000000000004, 14.500000000000004, 15.000000000000005, 15.500000000000005, 16.000000000000004, 16.500000000000004, 17.000000000000004, 17.500000000000004, 18.000000000000007, 18.500000000000004, 19.000000000000004, 19.500000000000004, 20.000000000000007, 20.500000000000007, 21.000000000000004, 21.500000000000007, 22.000000000000007, 22.500000000000004, 23.000000000000

In [7]:
len(amplitude)*len(width)*len(lag)*len(Sims)

336000

In [8]:
titles = [r'Field $\phi(x)$', r'Smoothed $\phi(x)$']
file_plots = '/home/dpirvu/collisions/plots/'
plot_sim_suffix = '_for_t'+str(nTime)+'_x'+str(nLat)+'_fields'

## Extract Data from Simulations 

In [9]:
def extract_data(filename, col):
    infile = open(filename,'r')
    lines = infile.readlines()
    field_values = [float(line.split()[col]) for line in lines[2:]]
    infile.close()
    return field_values

def truncateNum(num, decimal_places):
    StrNum = str(num)
    p = StrNum.find(".") + 1 + decimal_places
    return ( StrNum[0:p] )
    
def sim_location(x, y, lag, sim):
    """ x = width, y = amplitude """
    return '/gpfs/dpirvu/collisions/small_fluct_t'+str(nTime)+'_x'+str(nLat)+'_width{:.8f}'.format(x)+'_amp{:.8f}'.format(y)+'_lag'+str(lag)+'_sim'+str(sim)+'_fields.dat'

def sim_suffix(col, x, y, lag, sim):
    return '_col'+str(col)+'_lag'+str(lag)+'_width{:.3f}'.format(x)+'_amp{:.3f}'.format(y)+'_t'+str(nTime)+'_x'+str(nLat)+'_sim'+str(sim)+'_fields'

def simulation(col, x, y, lag, sim):
    simulation = extract_data(sim_location(x, y, lag, sim), col)
    return np.reshape(simulation, (nTime, nLat))

## Plot Spacetime Diagrams

In [10]:
def plot_data_from_files(col, x, y, lag, sim):
    fig = plt.figure(figsize=(1,1))
    tableau = simulation(col, x, y, lag, sim)
    im = plt.imshow(tableau, aspect='auto', origin='lower')
#    plt.title(titles[col]+': wid = {:.3f}'.format(x)+', amp = {:.3f}'.format(y)+', lag = '+str(lag)+', sim = '+str(sim))
    plt.colorbar(im); plt.show()
#    plt.savefig(field_plots + 'field' + sim_suffix(col, x, y, lag, sim) + '.png')
    return

def plot_window_from_files(col, x, y, lag, sim):
    plt.figure(figsize=(1,1))
    field = simulation(col, x, y, lag, sim)[int(nTime/2.) : int(nTime-(tean2-tean1)/2.), int(mean1) : int(mean2)]
    im = plt.imshow(field, aspect='auto', origin='lower', extent=[int(mean1), int(mean2), int(nTime/2.), int(nTime-(tean2-tean1)/2.)])
#    plt.title(titles[col]+': wid = {:.3f}'.format(x)+', amp = {:.3f}'.format(y)+', lag = '+str(lag)+', sim = '+str(sim))
    plt.colorbar(im); plt.show()
#    plt.savefig(field_plots + 'window_on_field' + sim_suffix(col, x, y, lag, sim) + '.png')
    return

## Mask Field 

In [11]:
def add_mask(col, x, y, lag, sim, threshold):
    masked_field = []
    mask = np.zeros((nTime, nLat))
    field = simulation(col, x, y, lag, sim)
    for i in range(nLat):
        for j in range(nTime):
            if field[j,i] >= np.pi+threshold or field[j,i] <= np.pi-threshold:
                mask[j,i] = 1
    return field * mask

def plot_masked_window(col, x, y, lag, sim, threshold):
    plt.figure(figsize=(3, 3))
    masked_field = add_mask(col, x, y, lag, sim, threshold)[int(nTime/2.) : int(nTime-(tean2-tean1)/2.), int(mean1) : int(mean2)]
    im = plt.imshow(masked_field, aspect='auto', origin='lower', extent=[int(mean1), int(mean2), int(nTime/2.), int(nTime-(tean2-tean1)/2.)])
    plt.xlabel(r'$dx$'); plt.ylabel(r'$dt$')
    plt.title('Masked '+titles[col]+': wid = {:.3f}'.format(x)+', amp = {:.3f}'.format(y)+', lag = '+str(lag)+', sim = '+str(sim))
    plt.colorbar(im); plt.show()
#    plt.savefig(field_plots + 'masked_field' + sim_suffix(col, x, y, lag, sim) + '.png')
    return

## Decay Test 

In [13]:
import matplotlib
matplotlib.rcParams.update({'font.size': 20})

def contour_test(col, x, y, lag, threshold):
    window_size = ( int(nTime-(tean2-tean1)/2.) - int(nTime/2.) ) * (mean2 - mean1)
    counter = 0.
    field_2wp = add_mask(col, x, y, lag, 5461, threshold)[int(nTime/2.) : int(nTime-(tean2-tean1)/2.), int(mean1) : int(mean2)] 
    if np.count_nonzero(field_2wp) > 0.1*window_size:
        counter = 1.
    return counter

def decay_test(col, x, y, lag, threshold):
    window_size = ( int(nTime-(tean2-tean1)/2.) - int(nTime/2.) ) * (mean2 - mean1)
    counter = 0.
    for sim in Sims:
            field_fluct = add_mask(col, x, y, lag, sim, threshold)[int(nTime/2.) : int(nTime-(tean2-tean1)/2.), int(mean1) : int(mean2)]
            if np.count_nonzero(field_fluct) > 0.1*window_size:
                counter += 1.
    return counter/len(Sims)

def plot_decay_rate(col, lagList, threshold):
    fig = plt.subplots(figsize=(9*2, 9))
    gs = gridspec.GridSpec(1, 2, width_ratios=[4.3, 5]) 
    ax0 = plt.subplot(gs[0])
    ax1 = plt.subplot(gs[1], sharey=ax0)
    plt.setp(ax1.get_yticklabels(), visible=False)
    plt.subplots_adjust(wspace=.05)

    x, y = np.meshgrid(dx*np.asarray(width), np.asarray(amplitude))
  
    stats1 = [[decay_test(col, x, y, lagList[0], threshold) for x in width] for y in amplitude]
    ax0.imshow(stats1, aspect='auto', cmap=plt.cm.get_cmap('plasma', 21), origin='lower', extent=[dx*width[0], dx*width[-1], amplitude[0], amplitude[-1]])

    stats2 = [[decay_test(col, x, y, lagList[1], threshold) for x in width] for y in amplitude]
    im = ax1.imshow(stats2, aspect='auto', cmap=plt.cm.get_cmap('plasma', 21), origin='lower', extent=[dx*width[0], dx*width[-1], amplitude[0], amplitude[-1]])

    ax0.set(xlabel = r'$\epsilon$', ylabel = r'$A$')
    ax1.set(xlabel = r'$\epsilon$')

    contour1 = [[contour_test(col, x, y, lagList[0], threshold) for x in width] for y in amplitude]
    ax0.contour(x, y, contour1, levels=[0.5, 1.5], colors=('k',), linewidths=(1.5,), origin='lower')
    contour2 = [[contour_test(col, x, y, lagList[1], threshold) for x in width] for y in amplitude]
    ax1.contour(x, y, contour2, levels=[0.5, 1.5], colors=('k',), linewidths=(1.5,), origin='lower')

    plt.colorbar(im, ticks = np.arange(21), drawedges='True')
    plt.savefig(file_plots+'decay_test_fluct_thr'+str(threshold)+'.png')
    plt.show()
    
    print('stats1 = ', stats1)
    print('stats2 = ', stats2)
    return np.count_nonzero(stats1), np.count_nonzero(stats1 == 1.), np.count_nonzero(stats2), np.count_nonzero(stats2 == 1.)

In [ ]:
np.asarray(plot_decay_rate(1, lag, np.pi/2))

In [26]:
stats1 =  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.1], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.1, 0.0, 0.05, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.05, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.05, 0.05, 0.0, 0.05, 0.0, 0.05], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.05, 0.15, 0.05, 0.05, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.05, 0.05, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.05, 0.0, 0.0, 0.05, 0.05, 0.1, 0.1, 0.1, 0.0, 0.15, 0.15, 0.1, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.05, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.05, 0.05, 0.05, 0.0, 0.0, 0.05, 0.05, 0.1, 0.0, 0.0, 0.1, 0.05, 0.0, 0.0, 0.1, 0.05, 0.05, 0.1, 0.1, 0.25, 0.2], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.05, 0.0, 0.05, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.1, 0.15, 0.0, 0.05, 0.05, 0.2, 0.0, 0.1, 0.15, 0.0, 0.15, 0.05, 0.15, 0.45, 0.15, 0.2, 0.2, 0.1, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.05, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.05, 0.0, 0.05, 0.1, 0.0, 0.05, 0.05, 0.0, 0.1, 0.0, 0.05, 0.05, 0.1, 0.05, 0.1, 0.05, 0.1, 0.1, 0.15, 0.1, 0.25, 0.05, 0.35, 0.1, 0.2, 0.05, 0.1, 0.25, 0.05, 0.15, 0.2, 0.2], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.05, 0.0, 0.05, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.05, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.15, 0.05, 0.1, 0.0, 0.0, 0.05, 0.2, 0.1, 0.1, 0.15, 0.2, 0.15, 0.2, 0.3, 0.25, 0.3, 0.45, 0.2, 0.2, 0.5, 0.45, 0.2], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.05, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.1, 0.05, 0.0, 0.05, 0.05, 0.15, 0.0, 0.1, 0.05, 0.05, 0.1, 0.1, 0.15, 0.15, 0.2, 0.3, 0.3, 0.3, 0.2, 0.15, 0.2, 0.2, 0.3, 0.25, 0.4, 0.35, 0.4, 0.5], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.05, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.1, 0.1, 0.0, 0.05, 0.0, 0.05, 0.0, 0.0, 0.0, 0.05, 0.0, 0.15, 0.15, 0.05, 0.0, 0.0, 0.15, 0.0, 0.05, 0.05, 0.05, 0.15, 0.1, 0.15, 0.3, 0.2, 0.15, 0.35, 0.35, 0.2, 0.3, 0.3, 0.2, 0.35, 0.35, 0.35, 0.3, 0.45, 0.35, 0.5, 0.6, 0.6], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.05, 0.05, 0.0, 0.15, 0.05, 0.05, 0.05, 0.0, 0.0, 0.0, 0.1, 0.15, 0.1, 0.1, 0.1, 0.1, 0.1, 0.25, 0.25, 0.25, 0.35, 0.35, 0.25, 0.15, 0.05, 0.3, 0.3, 0.15, 0.4, 0.25, 0.3, 0.3, 0.35, 0.45, 0.55, 0.6, 0.55, 0.5, 0.6, 0.55, 0.55], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.05, 0.0, 0.1, 0.0, 0.0, 0.0, 0.05, 0.05, 0.1, 0.05, 0.0, 0.05, 0.05, 0.0, 0.15, 0.2, 0.05, 0.15, 0.1, 0.1, 0.05, 0.2, 0.2, 0.15, 0.1, 0.15, 0.15, 0.2, 0.2, 0.1, 0.15, 0.2, 0.25, 0.3, 0.5, 0.35, 0.4, 0.35, 0.4, 0.45, 0.55, 0.6, 0.4, 0.55, 0.55, 0.5, 0.75, 0.65, 0.6], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.05, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.05, 0.0, 0.05, 0.0, 0.15, 0.05, 0.05, 0.0, 0.05, 0.1, 0.05, 0.1, 0.2, 0.0, 0.05, 0.05, 0.1, 0.2, 0.15, 0.15, 0.3, 0.25, 0.15, 0.2, 0.2, 0.15, 0.35, 0.25, 0.2, 0.2, 0.3, 0.35, 0.3, 0.5, 0.4, 0.35, 0.6, 0.65, 0.55, 0.65, 0.45, 0.45, 0.45, 0.5, 0.6, 0.65, 0.8, 0.75], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.05, 0.1, 0.1, 0.05, 0.1, 0.05, 0.05, 0.25, 0.15, 0.15, 0.2, 0.3, 0.1, 0.15, 0.2, 0.35, 0.25, 0.3, 0.25, 0.3, 0.25, 0.3, 0.1, 0.55, 0.3, 0.3, 0.35, 0.35, 0.5, 0.75, 0.6, 0.8, 0.5, 0.6, 0.75, 0.4, 0.75, 0.7, 0.75, 0.6, 0.8, 0.75], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.05, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.1, 0.1, 0.0, 0.15, 0.1, 0.05, 0.1, 0.2, 0.15, 0.1, 0.05, 0.2, 0.15, 0.35, 0.25, 0.35, 0.2, 0.05, 0.25, 0.3, 0.5, 0.15, 0.45, 0.5, 0.3, 0.4, 0.35, 0.5, 0.4, 0.5, 0.55, 0.5, 0.45, 0.4, 0.8, 0.5, 0.55, 0.6, 0.6, 0.75, 0.8, 0.7, 0.95, 0.65, 0.8, 0.65, 0.85], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.1, 0.0, 0.0, 0.05, 0.0, 0.1, 0.05, 0.15, 0.05, 0.05, 0.05, 0.05, 0.1, 0.05, 0.1, 0.1, 0.15, 0.15, 0.15, 0.15, 0.05, 0.1, 0.25, 0.1, 0.15, 0.25, 0.3, 0.15, 0.3, 0.55, 0.15, 0.4, 0.2, 0.45, 0.45, 0.5, 0.4, 0.5, 0.65, 0.45, 0.65, 0.65, 0.6, 0.5, 0.8, 0.7, 0.7, 0.6, 0.9, 0.55, 0.7, 0.7, 0.65, 1.0, 0.9, 0.75], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.05, 0.05, 0.0, 0.05, 0.0, 0.05, 0.0, 0.05, 0.0, 0.05, 0.05, 0.05, 0.25, 0.1, 0.15, 0.1, 0.0, 0.1, 0.15, 0.15, 0.25, 0.15, 0.2, 0.1, 0.1, 0.1, 0.35, 0.25, 0.15, 0.35, 0.4, 0.5, 0.55, 0.5, 0.5, 0.4, 0.5, 0.6, 0.4, 0.45, 0.65, 0.75, 0.5, 0.6, 0.65, 0.8, 0.7, 0.7, 0.8, 0.6, 0.85, 0.85, 0.75, 0.8, 0.85, 0.85, 0.95, 0.9, 0.8], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.05, 0.0, 0.1, 0.0, 0.05, 0.05, 0.05, 0.0, 0.15, 0.05, 0.1, 0.1, 0.15, 0.25, 0.05, 0.1, 0.25, 0.3, 0.2, 0.15, 0.2, 0.2, 0.15, 0.35, 0.45, 0.35, 0.2, 0.25, 0.35, 0.25, 0.35, 0.3, 0.45, 0.3, 0.7, 0.45, 0.6, 0.75, 0.9, 0.7, 0.6, 0.55, 0.55, 0.85, 0.6, 0.75, 0.85, 0.8, 0.7, 0.85, 0.7, 0.8, 0.8, 0.75, 0.8, 0.65, 0.95, 1.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.05, 0.0, 0.05, 0.0, 0.0, 0.15, 0.05, 0.05, 0.05, 0.05, 0.1, 0.05, 0.05, 0.1, 0.25, 0.2, 0.0, 0.05, 0.25, 0.15, 0.2, 0.35, 0.2, 0.25, 0.25, 0.2, 0.05, 0.3, 0.35, 0.25, 0.5, 0.35, 0.3, 0.45, 0.65, 0.5, 0.5, 0.5, 0.5, 0.7, 0.6, 0.65, 0.5, 0.7, 0.65, 0.7, 0.85, 0.85, 0.8, 0.85, 0.85, 0.95, 0.85, 0.8, 0.85, 0.85, 0.95, 1.0, 0.95, 1.0, 1.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.05, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.05, 0.05, 0.1, 0.05, 0.1, 0.2, 0.1, 0.0, 0.05, 0.1, 0.2, 0.2, 0.25, 0.05, 0.1, 0.05, 0.1, 0.25, 0.25, 0.15, 0.2, 0.35, 0.35, 0.2, 0.55, 0.35, 0.3, 0.5, 0.35, 0.3, 0.45, 0.4, 0.3, 0.5, 0.65, 0.65, 0.65, 0.65, 0.75, 0.75, 0.85, 0.65, 0.7, 0.8, 0.8, 0.95, 0.85, 0.75, 0.9, 0.9, 0.8, 1.0, 0.9, 1.0, 0.85, 0.85, 0.9, 0.9, 1.0, 0.95], [0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.1, 0.05, 0.0, 0.1, 0.05, 0.15, 0.1, 0.05, 0.1, 0.1, 0.2, 0.15, 0.1, 0.2, 0.25, 0.1, 0.1, 0.2, 0.3, 0.1, 0.1, 0.35, 0.2, 0.35, 0.4, 0.4, 0.45, 0.2, 0.35, 0.45, 0.45, 0.55, 0.55, 0.55, 0.55, 0.75, 0.55, 0.7, 0.65, 0.6, 0.7, 0.55, 0.75, 0.65, 0.8, 0.75, 0.8, 0.95, 0.7, 1.0, 0.9, 0.95, 0.9, 1.0, 0.8, 0.95, 0.95, 0.95, 0.85, 0.95, 0.95, 1.0, 0.95, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.05, 0.0, 0.0, 0.0, 0.1, 0.05, 0.15, 0.05, 0.1, 0.1, 0.1, 0.3, 0.1, 0.0, 0.15, 0.05, 0.15, 0.15, 0.1, 0.0, 0.2, 0.3, 0.3, 0.4, 0.35, 0.3, 0.2, 0.45, 0.3, 0.5, 0.3, 0.4, 0.6, 0.7, 0.65, 0.5, 0.45, 0.65, 0.65, 0.75, 0.8, 0.7, 0.7, 0.75, 0.8, 0.95, 0.85, 0.8, 0.8, 0.85, 1.0, 0.8, 0.95, 0.9, 0.85, 0.9, 0.8, 0.95, 1.0, 0.95, 0.95, 1.0, 1.0, 0.9, 1.0, 1.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.05, 0.0, 0.0, 0.0, 0.05, 0.05, 0.0, 0.05, 0.05, 0.1, 0.05, 0.1, 0.05, 0.0, 0.1, 0.05, 0.1, 0.05, 0.15, 0.2, 0.1, 0.25, 0.1, 0.4, 0.3, 0.35, 0.15, 0.15, 0.3, 0.2, 0.35, 0.25, 0.35, 0.3, 0.35, 0.35, 0.4, 0.6, 0.6, 0.55, 0.55, 0.6, 0.8, 0.75, 0.75, 0.85, 0.6, 0.75, 0.85, 0.7, 0.8, 0.85, 0.7, 0.95, 0.8, 0.85, 0.9, 0.9, 0.9, 0.95, 1.0, 0.9, 0.9, 0.9, 0.95, 0.95, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0], [0.0, 0.0, 0.0, 0.05, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.05, 0.05, 0.05, 0.05, 0.05, 0.0, 0.05, 0.05, 0.05, 0.0, 0.1, 0.05, 0.25, 0.1, 0.15, 0.15, 0.15, 0.1, 0.1, 0.15, 0.2, 0.2, 0.25, 0.4, 0.4, 0.4, 0.15, 0.25, 0.25, 0.6, 0.45, 0.55, 0.6, 0.6, 0.7, 0.65, 0.3, 0.65, 0.65, 0.55, 0.7, 0.8, 0.7, 0.8, 0.7, 0.85, 0.95, 0.9, 0.7, 0.95, 0.9, 0.9, 0.95, 0.9, 0.95, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 0.95, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.05, 0.05, 0.05, 0.05, 0.05, 0.1, 0.15, 0.1, 0.05, 0.15, 0.15, 0.15, 0.15, 0.05, 0.4, 0.15, 0.25, 0.05, 0.4, 0.2, 0.15, 0.25, 0.25, 0.25, 0.25, 0.45, 0.45, 0.25, 0.3, 0.55, 0.65, 0.65, 0.65, 0.55, 0.6, 0.75, 0.5, 0.75, 0.8, 0.9, 0.7, 0.75, 0.8, 0.8, 0.8, 0.9, 0.8, 0.9, 1.0, 0.9, 1.0, 0.9, 0.9, 0.9, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 0.9, 1.0, 0.95, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.05, 0.05, 0.0, 0.05, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.05, 0.05, 0.05, 0.05, 0.15, 0.05, 0.1, 0.2, 0.1, 0.1, 0.25, 0.15, 0.3, 0.2, 0.2, 0.2, 0.25, 0.3, 0.35, 0.35, 0.4, 0.45, 0.35, 0.4, 0.2, 0.5, 0.5, 0.7, 0.5, 0.7, 0.6, 0.5, 0.95, 0.65, 0.8, 0.8, 0.6, 0.7, 0.8, 0.7, 1.0, 0.95, 0.9, 0.95, 0.95, 0.9, 1.0, 1.0, 0.95, 0.95, 0.9, 1.0, 1.0, 0.95, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.0, 0.0, 0.0, 0.05, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.05, 0.15, 0.05, 0.0, 0.0, 0.05, 0.05, 0.15, 0.15, 0.1, 0.25, 0.25, 0.35, 0.2, 0.35, 0.5, 0.3, 0.4, 0.15, 0.3, 0.4, 0.35, 0.45, 0.4, 0.6, 0.6, 0.3, 0.7, 0.55, 0.5, 0.65, 0.6, 0.65, 0.75, 0.7, 0.65, 0.8, 0.9, 0.8, 0.9, 0.85, 0.9, 0.9, 1.0, 0.95, 1.0, 1.0, 0.95, 0.9, 1.0, 0.95, 1.0, 1.0, 1.0, 0.95, 0.95, 1.0, 1.0, 0.95, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.2, 0.05, 0.15, 0.1, 0.05, 0.2, 0.1, 0.1, 0.15, 0.3, 0.35, 0.25, 0.25, 0.3, 0.2, 0.3, 0.45, 0.5, 0.6, 0.45, 0.45, 0.4, 0.45, 0.55, 0.6, 0.7, 0.55, 0.65, 0.7, 0.85, 0.75, 0.85, 0.75, 0.7, 0.85, 0.75, 0.6, 0.95, 0.8, 0.75, 0.95, 0.95, 0.9, 1.0, 0.95, 1.0, 1.0, 0.95, 0.95, 0.85, 0.95, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.05, 0.0, 0.1, 0.15, 0.05, 0.0, 0.05, 0.05, 0.15, 0.1, 0.1, 0.05, 0.1, 0.25, 0.2, 0.3, 0.3, 0.2, 0.25, 0.1, 0.35, 0.45, 0.2, 0.55, 0.35, 0.2, 0.65, 0.45, 0.6, 0.15, 0.55, 0.5, 0.6, 0.65, 0.65, 0.55, 0.75, 0.55, 0.75, 0.7, 0.7, 0.85, 0.85, 0.9, 0.95, 1.0, 0.8, 0.95, 1.0, 1.0, 0.9, 0.95, 1.0, 1.0, 0.95, 1.0, 0.9, 1.0, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.05, 0.05, 0.0, 0.1, 0.0, 0.1, 0.15, 0.1, 0.05, 0.05, 0.0, 0.05, 0.1, 0.25, 0.3, 0.2, 0.15, 0.3, 0.25, 0.4, 0.35, 0.4, 0.3, 0.3, 0.65, 0.65, 0.65, 0.6, 0.6, 0.65, 0.8, 0.55, 0.65, 0.65, 0.5, 0.9, 0.8, 0.85, 0.7, 1.0, 0.95, 0.9, 0.95, 0.8, 0.95, 1.0, 1.0, 1.0, 1.0, 0.9, 0.95, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.0, 0.0, 0.05, 0.05, 0.05, 0.0, 0.0, 0.2, 0.05, 0.05, 0.05, 0.05, 0.15, 0.2, 0.05, 0.15, 0.1, 0.0, 0.2, 0.15, 0.1, 0.25, 0.3, 0.3, 0.2, 0.15, 0.45, 0.4, 0.4, 0.45, 0.45, 0.45, 0.65, 0.5, 0.5, 0.6, 0.4, 0.6, 0.6, 0.8, 0.7, 0.8, 0.75, 0.7, 0.9, 0.95, 0.65, 0.9, 1.0, 0.85, 1.0, 0.9, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 0.9, 1.0, 0.95, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.05, 0.15, 0.0, 0.0, 0.05, 0.1, 0.15, 0.05, 0.1, 0.25, 0.2, 0.2, 0.25, 0.25, 0.2, 0.25, 0.2, 0.35, 0.35, 0.45, 0.5, 0.25, 0.5, 0.45, 0.45, 0.55, 0.5, 0.85, 0.6, 0.55, 0.65, 0.75, 0.9, 0.7, 0.75, 0.7, 0.8, 0.85, 1.0, 0.95, 0.95, 0.95, 1.0, 0.95, 0.9, 1.0, 1.0, 0.95, 0.9, 1.0, 0.95, 0.9, 0.95, 1.0, 0.95, 1.0, 0.95, 1.0, 1.0, 1.0, 0.95, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.05, 0.05, 0.0, 0.05, 0.05, 0.1, 0.05, 0.0, 0.0, 0.05, 0.05, 0.05, 0.05, 0.2, 0.15, 0.3, 0.45, 0.35, 0.35, 0.35, 0.55, 0.4, 0.6, 0.5, 0.5, 0.4, 0.3, 0.65, 0.45, 0.7, 0.5, 0.65, 0.7, 0.75, 0.95, 0.75, 0.85, 0.9, 0.9, 0.85, 0.95, 0.9, 0.85, 1.0, 0.9, 0.8, 0.85, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.1, 0.2, 0.1, 0.2, 0.2, 0.2, 0.25, 0.2, 0.25, 0.15, 0.2, 0.3, 0.15, 0.2, 0.5, 0.5, 0.6, 0.6, 0.5, 0.3, 0.7, 0.4, 0.7, 0.9, 0.7, 0.65, 0.65, 0.65, 0.85, 0.8, 0.9, 0.9, 0.7, 0.95, 0.8, 0.85, 0.85, 0.9, 1.0, 1.0, 0.95, 1.0, 0.9, 1.0, 0.95, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.05, 0.15, 0.05, 0.15, 0.1, 0.0, 0.1, 0.1, 0.2, 0.1, 0.3, 0.3, 0.25, 0.2, 0.35, 0.25, 0.45, 0.25, 0.4, 0.25, 0.55, 0.55, 0.4, 0.6, 0.5, 0.7, 0.5, 0.55, 0.65, 0.7, 0.7, 0.7, 0.8, 0.85, 0.85, 0.95, 0.95, 0.95, 0.95, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.0, 0.05, 0.05, 0.0, 0.05, 0.1, 0.05, 0.1, 0.1, 0.0, 0.05, 0.05, 0.05, 0.1, 0.2, 0.25, 0.25, 0.35, 0.35, 0.2, 0.5, 0.3, 0.3, 0.65, 0.65, 0.65, 0.8, 0.7, 0.7, 0.55, 0.8, 0.6, 0.75, 0.75, 0.75, 0.7, 0.9, 0.9, 0.85, 0.9, 0.85, 0.9, 0.95, 0.95, 0.9, 1.0, 0.9, 0.9, 0.95, 1.0, 0.95, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.0, 0.0, 0.1, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.15, 0.05, 0.1, 0.05, 0.2, 0.1, 0.2, 0.15, 0.2, 0.5, 0.35, 0.35, 0.4, 0.5, 0.5, 0.35, 0.65, 0.6, 0.65, 0.65, 0.8, 0.8, 0.8, 0.75, 0.75, 0.95, 0.75, 0.85, 0.9, 0.7, 0.95, 0.85, 0.85, 1.0, 0.95, 0.95, 0.9, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 0.95, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.0, 0.05, 0.0, 0.05, 0.0, 0.0, 0.0, 0.15, 0.0, 0.05, 0.1, 0.15, 0.1, 0.05, 0.15, 0.25, 0.45, 0.35, 0.25, 0.55, 0.5, 0.45, 0.6, 0.75, 0.65, 0.4, 0.65, 0.7, 0.7, 0.8, 0.85, 0.85, 0.75, 0.85, 0.95, 1.0, 0.85, 0.8, 0.95, 0.85, 1.0, 0.9, 0.95, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.0, 0.0, 0.0, 0.1, 0.1, 0.0, 0.15, 0.1, 0.1, 0.2, 0.2, 0.2, 0.2, 0.15, 0.2, 0.15, 0.3, 0.4, 0.45, 0.7, 0.65, 0.55, 0.45, 0.8, 0.5, 0.4, 0.7, 0.8, 0.8, 0.85, 0.75, 1.0, 0.8, 1.0, 0.75, 0.9, 0.8, 0.9, 1.0, 0.95, 1.0, 0.95, 1.0, 1.0, 1.0, 0.95, 1.0, 0.95, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.05, 0.05, 0.0, 0.1, 0.05, 0.05, 0.05, 0.1, 0.1, 0.05, 0.3, 0.3, 0.15, 0.4, 0.1, 0.2, 0.25, 0.45, 0.35, 0.45, 0.3, 0.55, 0.7, 0.65, 0.75, 0.8, 0.65, 0.75, 0.8, 0.75, 0.75, 0.9, 0.95, 0.85, 0.8, 0.95, 0.95, 1.0, 0.95, 1.0, 1.0, 0.95, 0.9, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.15, 0.05, 0.05, 0.05, 0.15, 0.2, 0.15, 0.1, 0.25, 0.5, 0.4, 0.5, 0.55, 0.5, 0.5, 0.65, 0.65, 0.55, 0.85, 0.75, 0.8, 0.85, 0.95, 0.95, 0.9, 0.85, 1.0, 0.75, 0.95, 0.95, 0.95, 0.95, 0.85, 0.95, 1.0, 0.95, 1.0, 1.0, 0.95, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.05, 0.0, 0.0, 0.1, 0.15, 0.1, 0.1, 0.2, 0.1, 0.15, 0.3, 0.25, 0.2, 0.3, 0.2, 0.3, 0.75, 0.55, 0.5, 0.45, 0.55, 0.5, 0.65, 0.65, 0.8, 0.7, 0.75, 0.8, 1.0, 0.95, 0.95, 0.85, 0.8, 0.8, 1.0, 0.95, 1.0, 1.0, 1.0, 0.95, 1.0, 0.95, 1.0, 0.95, 0.9, 0.9, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.0, 0.0, 0.05, 0.05, 0.1, 0.0, 0.2, 0.1, 0.15, 0.1, 0.25, 0.3, 0.35, 0.35, 0.2, 0.4, 0.4, 0.55, 0.4, 0.6, 0.65, 0.7, 0.55, 0.9, 0.8, 0.8, 0.85, 0.8, 0.85, 0.85, 1.0, 0.9, 0.85, 0.95, 0.95, 0.95, 0.9, 0.95, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.1, 0.0, 0.05, 0.05, 0.15, 0.2, 0.35, 0.05, 0.15, 0.25, 0.25, 0.2, 0.4, 0.25, 0.3, 0.5, 0.4, 0.5, 0.55, 0.6, 0.7, 0.7, 0.8, 0.6, 0.7, 0.75, 0.85, 0.9, 0.85, 0.75, 1.0, 0.85, 0.85, 0.95, 0.85, 1.0, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 0.95, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.0, 0.05, 0.05, 0.1, 0.15, 0.05, 0.3, 0.05, 0.05, 0.15, 0.25, 0.25, 0.5, 0.3, 0.55, 0.6, 0.55, 0.45, 0.6, 0.7, 0.6, 0.7, 0.75, 0.7, 0.85, 0.9, 0.8, 0.95, 0.9, 0.85, 0.9, 1.0, 0.9, 0.85, 1.0, 0.95, 0.9, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.05, 0.0, 0.05, 0.1, 0.1, 0.05, 0.15, 0.35, 0.25, 0.25, 0.45, 0.35, 0.15, 0.45, 0.45, 0.45, 0.55, 0.65, 0.9, 0.6, 0.75, 0.7, 0.7, 0.75, 0.8, 0.95, 0.95, 0.85, 1.0, 1.0, 1.0, 0.85, 0.95, 1.0, 1.0, 0.9, 0.95, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.05, 0.1, 0.05, 0.15, 0.15, 0.2, 0.2, 0.25, 0.25, 0.35, 0.5, 0.3, 0.4, 0.45, 0.5, 0.7, 0.7, 0.55, 0.75, 0.75, 0.65, 0.95, 0.85, 0.7, 0.9, 0.95, 0.9, 0.8, 0.95, 0.95, 0.95, 0.95, 0.9, 1.0, 0.9, 0.95, 1.0, 0.95, 1.0, 0.95, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.05, 0.1, 0.1, 0.1, 0.15, 0.1, 0.4, 0.05, 0.2, 0.25, 0.25, 0.3, 0.5, 0.4, 0.5, 0.65, 0.75, 0.45, 0.65, 0.8, 0.8, 0.85, 0.9, 0.9, 0.85, 0.9, 0.9, 0.85, 0.85, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.1, 0.05, 0.05, 0.15, 0.25, 0.0, 0.2, 0.35, 0.25, 0.35, 0.35, 0.3, 0.45, 0.75, 0.65, 0.7, 0.6, 0.75, 0.75, 0.7, 0.85, 0.85, 0.85, 0.95, 0.75, 0.95, 0.9, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9, 1.0, 1.0, 0.9, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.15, 0.05, 0.1, 0.15, 0.35, 0.25, 0.5, 0.25, 0.15, 0.3, 0.55, 0.4, 0.45, 0.65, 0.5, 0.75, 0.65, 0.8, 0.7, 0.9, 0.9, 0.9, 0.8, 0.8, 0.95, 1.0, 0.9, 1.0, 0.9, 0.95, 0.95, 1.0, 1.0, 1.0, 0.95, 0.9, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.1, 0.05, 0.1, 0.1, 0.15, 0.05, 0.15, 0.3, 0.3, 0.45, 0.5, 0.35, 0.65, 0.45, 0.55, 0.7, 0.8, 0.6, 0.85, 0.95, 0.9, 0.85, 0.8, 0.8, 0.95, 0.9, 0.95, 0.9, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.05, 0.1, 0.15, 0.15, 0.1, 0.2, 0.4, 0.45, 0.4, 0.4, 0.45, 0.35, 0.5, 0.6, 0.7, 0.5, 0.8, 0.65, 0.95, 0.75, 0.8, 0.9, 0.95, 0.9, 0.9, 1.0, 0.9, 1.0, 0.9, 0.95, 0.95, 0.95, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.95, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.15, 0.2, 0.05, 0.15, 0.4, 0.3, 0.35, 0.35, 0.55, 0.5, 0.45, 0.35, 0.55, 0.45, 0.55, 0.7, 0.65, 0.8, 0.75, 0.9, 0.75, 0.95, 0.9, 1.0, 1.0, 1.0, 0.9, 0.9, 0.95, 0.9, 1.0, 0.95, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.15, 0.15, 0.2, 0.15, 0.15, 0.25, 0.25, 0.2, 0.3, 0.55, 0.5, 0.6, 0.6, 0.9, 0.95, 0.7, 0.75, 0.7, 1.0, 0.9, 0.9, 0.9, 1.0, 0.95, 1.0, 0.95, 0.95, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.05, 0.1, 0.05, 0.2, 0.2, 0.4, 0.4, 0.4, 0.6, 0.6, 0.5, 0.6, 0.55, 0.6, 0.65, 0.7, 0.9, 0.85, 0.85, 1.0, 0.95, 0.9, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 0.9, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.1, 0.15, 0.2, 0.3, 0.25, 0.2, 0.3, 0.55, 0.4, 0.55, 0.45, 0.65, 0.75, 0.65, 0.95, 0.7, 1.0, 0.8, 0.75, 0.95, 0.95, 1.0, 0.9, 0.95, 0.95, 0.95, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.15, 0.15, 0.3, 0.2, 0.3, 0.4, 0.45, 0.55, 0.5, 0.65, 0.55, 0.6, 0.6, 0.85, 0.8, 0.85, 0.9, 0.9, 0.7, 1.0, 1.0, 1.0, 0.9, 1.0, 1.0, 1.0, 0.9, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.15, 0.2, 0.2, 0.3, 0.15, 0.5, 0.2, 0.5, 0.45, 0.6, 0.85, 0.65, 0.7, 0.7, 0.8, 0.85, 0.7, 0.9, 0.85, 1.0, 1.0, 0.95, 0.95, 0.9, 1.0, 1.0, 0.95, 1.0, 1.0, 0.95, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.2, 0.25, 0.15, 0.3, 0.25, 0.4, 0.4, 0.5, 0.55, 0.8, 0.7, 0.7, 0.8, 0.8, 0.7, 0.75, 0.95, 0.95, 1.0, 0.9, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.2, 0.25, 0.2, 0.25, 0.3, 0.3, 0.55, 0.55, 0.55, 0.7, 0.65, 0.8, 0.6, 0.75, 0.8, 0.9, 0.95, 0.9, 0.95, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.15, 0.3, 0.35, 0.35, 0.55, 0.5, 0.6, 0.6, 0.5, 0.75, 0.75, 0.55, 0.8, 0.85, 0.65, 0.95, 0.85, 0.95, 1.0, 0.95, 0.95, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.1, 0.15, 0.3, 0.25, 0.4, 0.3, 0.5, 0.55, 0.45, 0.7, 0.7, 0.95, 0.8, 1.0, 0.85, 0.85, 0.9, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.2, 0.1, 0.05, 0.4, 0.35, 0.45, 0.55, 0.65, 0.9, 0.65, 0.7, 0.65, 0.8, 0.8, 1.0, 0.8, 0.95, 1.0, 0.95, 0.95, 1.0, 0.95, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.3, 0.25, 0.3, 0.5, 0.6, 0.5, 0.75, 0.8, 0.7, 0.75, 0.7, 0.95, 0.8, 1.0, 0.8, 0.9, 0.9, 0.9, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.3, 0.2, 0.4, 0.45, 0.3, 0.35, 0.5, 0.85, 0.75, 0.9, 0.9, 0.85, 0.85, 0.8, 0.85, 1.0, 1.0, 0.95, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.3, 0.25, 0.3, 0.35, 0.55, 0.5, 0.75, 0.65, 0.8, 0.85, 0.85, 0.9, 0.95, 0.85, 1.0, 0.9, 1.0, 0.95, 0.95, 1.0, 0.95, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.25, 0.35, 0.4, 0.45, 0.4, 0.75, 0.65, 0.65, 0.85, 0.8, 0.9, 1.0, 0.9, 0.85, 1.0, 0.95, 0.9, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.05, 0.15, 0.35, 0.65, 0.55, 0.85, 0.7, 0.8, 0.95, 0.85, 0.8, 0.9, 0.75, 0.95, 0.95, 1.0, 0.95, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.2, 0.5, 0.4, 0.55, 0.4, 0.55, 0.6, 0.85, 0.8, 0.95, 0.85, 0.9, 0.95, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.3, 0.4, 0.45, 0.45, 0.75, 0.8, 0.75, 0.6, 0.7, 0.8, 0.8, 0.85, 0.9, 0.95, 0.95, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.25, 0.6, 0.5, 0.4, 0.5, 0.7, 0.7, 0.65, 0.85, 0.8, 0.9, 0.85, 1.0, 0.9, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.35, 0.25, 0.5, 0.5, 0.45, 0.75, 0.8, 0.8, 0.95, 0.95, 0.9, 1.0, 1.0, 0.85, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.4, 0.5, 0.45, 0.7, 0.6, 0.6, 0.9, 0.95, 0.85, 0.95, 0.95, 0.95, 0.95, 0.95, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.3, 0.6, 0.5, 0.5, 0.8, 0.6, 0.9, 0.8, 0.9, 0.9, 0.85, 0.95, 0.95, 1.0, 0.95, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.5, 0.65, 0.6, 0.75, 0.75, 0.8, 0.75, 0.85, 0.95, 0.95, 0.95, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.45, 0.5, 0.45, 0.7, 0.75, 0.75, 0.8, 0.85, 0.9, 0.8, 0.95, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.5, 0.65, 0.65, 0.8, 0.7, 0.9, 1.0, 0.9, 1.0, 1.0, 0.95, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.6, 0.35, 0.6, 0.75, 0.8, 0.8, 0.75, 0.95, 0.85, 0.9, 0.95, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.6, 0.5, 0.7, 0.6, 0.7, 0.85, 0.8, 0.9, 0.9, 1.0, 0.95, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.65, 0.6, 0.7, 0.75, 0.8, 0.85, 0.85, 0.85, 1.0, 0.95, 0.95, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.6, 0.55, 0.5, 0.75, 0.8, 0.8, 0.95, 1.0, 0.95, 0.95, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.75, 0.65, 0.75, 0.6, 0.85, 0.8, 0.95, 0.95, 0.95, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.65, 0.7, 0.65, 0.9, 0.8, 0.9, 1.0, 0.95, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.65, 0.8, 0.6, 0.85, 0.85, 0.95, 0.95, 0.85, 0.9, 1.0, 0.9, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.8, 0.8, 0.9, 0.75, 0.95, 0.95, 1.0, 1.0, 1.0, 1.0, 0.95, 0.95, 0.95, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.75, 0.85, 0.85, 0.75, 0.9, 0.85, 0.95, 0.95, 0.95, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.75, 0.7, 0.9, 0.9, 0.9, 0.85, 0.95, 1.0, 0.95, 1.0, 0.95, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.8, 0.9, 0.9, 0.95, 0.85, 0.95, 0.95, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]]
stats2 =  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.05, 0.05, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.05, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.05, 0.1, 0.0, 0.0, 0.05, 0.0, 0.05, 0.05], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.05, 0.0, 0.05, 0.05, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0], [0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.05, 0.0, 0.0, 0.0, 0.05, 0.05, 0.0, 0.0, 0.0, 0.05, 0.05, 0.0, 0.05, 0.05, 0.05, 0.0, 0.2, 0.0, 0.05, 0.0, 0.05, 0.0, 0.0, 0.05, 0.05, 0.1, 0.0, 0.05, 0.05], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.1, 0.0, 0.05, 0.0, 0.0, 0.0, 0.05, 0.05, 0.05, 0.05, 0.0, 0.05, 0.05, 0.1, 0.15, 0.1, 0.05, 0.0, 0.05, 0.05, 0.25, 0.1, 0.1, 0.05, 0.05, 0.0, 0.1, 0.25, 0.2, 0.05], [0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.05, 0.0, 0.05, 0.0, 0.05, 0.0, 0.0, 0.05, 0.05, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.1, 0.0, 0.05, 0.05, 0.0, 0.0, 0.0, 0.05, 0.1, 0.1, 0.0, 0.15, 0.1, 0.2, 0.05, 0.15, 0.1, 0.05, 0.15, 0.15, 0.1, 0.0, 0.05, 0.25, 0.1, 0.35, 0.1, 0.1, 0.3, 0.2, 0.15, 0.15], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.05, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.05, 0.05, 0.0, 0.0, 0.05, 0.0, 0.0, 0.1, 0.0, 0.05, 0.0, 0.0, 0.05, 0.0, 0.05, 0.0, 0.1, 0.05, 0.0, 0.0, 0.1, 0.1, 0.2, 0.05, 0.05, 0.15, 0.05, 0.3, 0.15, 0.05, 0.2, 0.0, 0.15, 0.1, 0.15, 0.25, 0.2, 0.25, 0.1, 0.25], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.05, 0.05, 0.0, 0.05, 0.0, 0.0, 0.0, 0.05, 0.0, 0.05, 0.05, 0.0, 0.05, 0.05, 0.05, 0.05, 0.05, 0.0, 0.05, 0.05, 0.0, 0.1, 0.05, 0.05, 0.0, 0.05, 0.1, 0.25, 0.15, 0.1, 0.05, 0.2, 0.0, 0.05, 0.15, 0.2, 0.2, 0.25, 0.15, 0.15, 0.25, 0.2, 0.25, 0.2, 0.25, 0.4, 0.3, 0.35, 0.3, 0.3], [0.0, 0.05, 0.0, 0.05, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.1, 0.05, 0.15, 0.05, 0.1, 0.0, 0.0, 0.05, 0.15, 0.2, 0.1, 0.05, 0.05, 0.15, 0.0, 0.15, 0.2, 0.0, 0.1, 0.2, 0.1, 0.2, 0.2, 0.25, 0.15, 0.35, 0.2, 0.25, 0.3, 0.1, 0.15, 0.15, 0.35, 0.2, 0.25, 0.3, 0.3, 0.45, 0.3, 0.2, 0.25], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.1, 0.0, 0.05, 0.0, 0.05, 0.0, 0.0, 0.05, 0.0, 0.05, 0.0, 0.05, 0.05, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.2, 0.1, 0.05, 0.05, 0.0, 0.05, 0.05, 0.15, 0.1, 0.05, 0.2, 0.15, 0.05, 0.15, 0.2, 0.15, 0.2, 0.1, 0.2, 0.15, 0.45, 0.2, 0.25, 0.35, 0.35, 0.25, 0.35, 0.4, 0.3, 0.3, 0.3, 0.15, 0.55, 0.55, 0.4, 0.45, 0.4, 0.45, 0.45], [0.0, 0.0, 0.0, 0.0, 0.05, 0.05, 0.0, 0.0, 0.05, 0.0, 0.0, 0.05, 0.0, 0.0, 0.05, 0.0, 0.0, 0.05, 0.0, 0.05, 0.0, 0.0, 0.05, 0.0, 0.05, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.1, 0.05, 0.1, 0.15, 0.0, 0.05, 0.05, 0.15, 0.15, 0.05, 0.1, 0.05, 0.15, 0.1, 0.15, 0.3, 0.15, 0.2, 0.25, 0.1, 0.2, 0.25, 0.15, 0.25, 0.5, 0.3, 0.3, 0.2, 0.25, 0.3, 0.5, 0.1, 0.5, 0.35, 0.55, 0.45, 0.5, 0.55, 0.45, 0.5, 0.7], [0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.1, 0.0, 0.05, 0.05, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.05, 0.0, 0.0, 0.05, 0.1, 0.05, 0.0, 0.05, 0.05, 0.0, 0.0, 0.05, 0.05, 0.0, 0.05, 0.15, 0.05, 0.1, 0.05, 0.2, 0.15, 0.15, 0.05, 0.15, 0.15, 0.25, 0.05, 0.1, 0.35, 0.25, 0.4, 0.4, 0.15, 0.2, 0.4, 0.4, 0.35, 0.35, 0.5, 0.4, 0.2, 0.55, 0.5, 0.55, 0.5, 0.4, 0.55, 0.35, 0.55, 0.65, 0.5, 0.6, 0.6, 0.55], [0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.05, 0.05, 0.0, 0.05, 0.05, 0.0, 0.05, 0.0, 0.15, 0.05, 0.05, 0.05, 0.0, 0.05, 0.0, 0.0, 0.05, 0.1, 0.05, 0.05, 0.05, 0.1, 0.0, 0.15, 0.15, 0.2, 0.05, 0.15, 0.15, 0.2, 0.25, 0.05, 0.05, 0.25, 0.3, 0.25, 0.15, 0.3, 0.15, 0.15, 0.3, 0.15, 0.4, 0.3, 0.35, 0.25, 0.5, 0.3, 0.5, 0.25, 0.35, 0.4, 0.25, 0.5, 0.4, 0.65, 0.55, 0.65, 0.8, 0.7, 0.75, 0.65, 0.8, 0.6, 0.55], [0.0, 0.0, 0.0, 0.05, 0.05, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.05, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.05, 0.05, 0.0, 0.05, 0.0, 0.05, 0.0, 0.05, 0.15, 0.1, 0.05, 0.2, 0.25, 0.15, 0.1, 0.1, 0.4, 0.1, 0.15, 0.2, 0.2, 0.15, 0.25, 0.15, 0.15, 0.25, 0.45, 0.25, 0.35, 0.25, 0.15, 0.25, 0.4, 0.45, 0.3, 0.6, 0.45, 0.35, 0.35, 0.4, 0.5, 0.55, 0.55, 0.5, 0.75, 0.7, 0.6, 0.55, 0.5, 0.55, 0.75, 0.8, 0.8, 0.75, 0.55, 0.75, 0.8], [0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.1, 0.15, 0.0, 0.0, 0.0, 0.1, 0.05, 0.1, 0.0, 0.05, 0.05, 0.15, 0.05, 0.0, 0.2, 0.1, 0.15, 0.05, 0.15, 0.2, 0.0, 0.1, 0.1, 0.1, 0.15, 0.25, 0.2, 0.1, 0.3, 0.05, 0.25, 0.25, 0.25, 0.2, 0.2, 0.2, 0.3, 0.45, 0.35, 0.25, 0.3, 0.4, 0.4, 0.25, 0.5, 0.6, 0.55, 0.55, 0.4, 0.6, 0.7, 0.5, 0.7, 0.65, 0.75, 0.65, 0.8, 0.85, 0.7, 0.85, 0.9, 0.8, 0.8, 0.7], [0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.05, 0.15, 0.0, 0.0, 0.05, 0.0, 0.1, 0.0, 0.05, 0.1, 0.15, 0.1, 0.1, 0.05, 0.0, 0.0, 0.1, 0.05, 0.05, 0.15, 0.15, 0.05, 0.1, 0.0, 0.1, 0.1, 0.15, 0.1, 0.1, 0.25, 0.2, 0.2, 0.1, 0.15, 0.15, 0.2, 0.3, 0.3, 0.1, 0.2, 0.3, 0.15, 0.4, 0.35, 0.4, 0.45, 0.6, 0.75, 0.35, 0.4, 0.4, 0.7, 0.55, 0.3, 0.4, 0.85, 0.55, 0.75, 0.8, 0.6, 0.6, 0.55, 0.65, 0.55, 0.85, 0.65, 0.8, 0.8, 0.75, 0.8, 0.85, 0.85, 0.9, 0.95], [0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.05, 0.05, 0.0, 0.0, 0.05, 0.05, 0.1, 0.0, 0.1, 0.05, 0.0, 0.15, 0.1, 0.0, 0.15, 0.15, 0.05, 0.1, 0.1, 0.1, 0.1, 0.25, 0.0, 0.15, 0.2, 0.15, 0.25, 0.1, 0.15, 0.25, 0.1, 0.3, 0.35, 0.4, 0.45, 0.35, 0.2, 0.35, 0.35, 0.55, 0.35, 0.4, 0.3, 0.5, 0.45, 0.4, 0.55, 0.55, 0.9, 0.45, 0.45, 0.6, 0.8, 0.95, 0.65, 0.75, 0.8, 0.55, 0.75, 0.85, 0.8, 0.9, 0.9, 0.7, 0.85, 0.9, 0.85, 0.7, 0.9, 0.85], [0.05, 0.0, 0.0, 0.05, 0.0, 0.0, 0.05, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.05, 0.0, 0.05, 0.1, 0.05, 0.1, 0.0, 0.0, 0.0, 0.1, 0.15, 0.0, 0.1, 0.05, 0.05, 0.15, 0.15, 0.1, 0.3, 0.05, 0.25, 0.25, 0.25, 0.35, 0.3, 0.2, 0.25, 0.35, 0.45, 0.35, 0.2, 0.3, 0.35, 0.45, 0.35, 0.45, 0.6, 0.65, 0.5, 0.85, 0.65, 0.5, 0.55, 0.6, 0.6, 0.7, 0.65, 0.75, 0.7, 0.55, 0.7, 0.55, 0.85, 0.65, 0.6, 0.8, 0.85, 0.8, 0.9, 0.75, 0.9, 0.8, 0.9, 1.0, 1.0, 0.9, 0.8], [0.0, 0.0, 0.0, 0.05, 0.05, 0.0, 0.0, 0.0, 0.05, 0.05, 0.1, 0.0, 0.0, 0.0, 0.1, 0.1, 0.1, 0.0, 0.1, 0.1, 0.25, 0.05, 0.15, 0.05, 0.0, 0.2, 0.25, 0.1, 0.2, 0.05, 0.35, 0.35, 0.05, 0.25, 0.25, 0.3, 0.25, 0.2, 0.35, 0.45, 0.35, 0.1, 0.45, 0.4, 0.55, 0.65, 0.55, 0.3, 0.55, 0.4, 0.3, 0.4, 0.65, 0.7, 0.75, 0.7, 0.65, 0.55, 0.5, 0.7, 0.65, 0.7, 0.7, 0.7, 0.7, 0.85, 0.7, 0.9, 0.7, 0.7, 0.95, 0.9, 0.9, 0.8, 0.85, 0.9, 0.95, 0.95, 0.8, 1.0], [0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.15, 0.1, 0.05, 0.05, 0.15, 0.1, 0.05, 0.0, 0.0, 0.2, 0.05, 0.2, 0.1, 0.1, 0.1, 0.1, 0.15, 0.1, 0.1, 0.05, 0.2, 0.25, 0.2, 0.15, 0.25, 0.2, 0.3, 0.25, 0.3, 0.35, 0.4, 0.45, 0.35, 0.35, 0.45, 0.3, 0.35, 0.65, 0.6, 0.5, 0.6, 0.6, 0.55, 0.5, 0.6, 0.6, 0.7, 0.8, 0.55, 0.85, 0.75, 0.7, 0.7, 0.75, 0.9, 0.8, 0.75, 0.75, 0.7, 0.9, 0.95, 0.9, 0.85, 0.7, 0.95, 0.9, 0.9, 0.95, 0.9, 0.95, 0.95, 0.9, 1.0, 0.85], [0.0, 0.0, 0.0, 0.1, 0.15, 0.1, 0.1, 0.0, 0.1, 0.15, 0.05, 0.05, 0.05, 0.0, 0.05, 0.0, 0.05, 0.15, 0.1, 0.05, 0.1, 0.2, 0.05, 0.1, 0.3, 0.15, 0.15, 0.25, 0.15, 0.25, 0.25, 0.25, 0.15, 0.45, 0.3, 0.35, 0.35, 0.45, 0.4, 0.35, 0.35, 0.65, 0.4, 0.4, 0.5, 0.55, 0.65, 0.5, 0.85, 0.65, 0.45, 0.65, 0.95, 0.65, 0.75, 0.9, 0.85, 0.85, 0.8, 0.8, 0.9, 0.85, 0.95, 0.9, 0.95, 0.95, 0.75, 0.9, 0.85, 0.95, 0.9, 0.85, 0.8, 1.0, 1.0, 0.95, 0.85, 1.0, 0.8, 0.95], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.05, 0.1, 0.05, 0.0, 0.05, 0.05, 0.1, 0.05, 0.15, 0.2, 0.15, 0.1, 0.1, 0.0, 0.2, 0.2, 0.2, 0.4, 0.15, 0.05, 0.25, 0.25, 0.15, 0.25, 0.2, 0.15, 0.25, 0.3, 0.45, 0.2, 0.45, 0.45, 0.55, 0.6, 0.45, 0.45, 0.5, 0.6, 0.55, 0.6, 0.65, 0.65, 0.6, 0.6, 0.75, 0.65, 0.75, 0.75, 0.95, 0.85, 0.75, 0.7, 0.9, 0.95, 0.8, 0.9, 0.8, 0.85, 0.85, 0.85, 0.95, 1.0, 1.0, 0.95, 1.0, 1.0, 0.9, 0.95, 0.9, 1.0, 0.9, 1.0, 1.0], [0.0, 0.0, 0.0, 0.05, 0.0, 0.1, 0.05, 0.0, 0.15, 0.1, 0.1, 0.2, 0.15, 0.1, 0.15, 0.05, 0.0, 0.1, 0.25, 0.2, 0.2, 0.1, 0.25, 0.15, 0.1, 0.25, 0.3, 0.3, 0.2, 0.25, 0.45, 0.6, 0.35, 0.35, 0.35, 0.45, 0.45, 0.5, 0.4, 0.75, 0.6, 0.5, 0.7, 0.65, 0.65, 0.75, 0.85, 0.7, 0.75, 0.5, 0.6, 0.75, 0.75, 0.9, 0.7, 0.7, 0.9, 0.9, 0.95, 0.8, 1.0, 0.8, 0.8, 1.0, 0.95, 0.9, 0.95, 0.9, 0.9, 0.95, 0.9, 1.0, 0.9, 0.95, 0.95, 0.95, 0.95, 1.0, 1.0, 0.95], [0.05, 0.05, 0.1, 0.1, 0.05, 0.0, 0.1, 0.05, 0.05, 0.3, 0.05, 0.05, 0.0, 0.0, 0.1, 0.2, 0.1, 0.25, 0.05, 0.45, 0.15, 0.4, 0.35, 0.4, 0.3, 0.4, 0.4, 0.25, 0.15, 0.25, 0.6, 0.45, 0.6, 0.45, 0.6, 0.6, 0.4, 0.4, 0.5, 0.7, 0.5, 0.65, 0.75, 0.75, 0.6, 0.8, 0.7, 0.9, 0.7, 0.85, 0.6, 0.8, 0.7, 0.9, 0.85, 0.8, 0.65, 1.0, 0.95, 0.9, 0.9, 0.95, 1.0, 1.0, 0.9, 1.0, 0.9, 0.95, 0.95, 1.0, 1.0, 1.0, 1.0, 0.95, 0.95, 1.0, 1.0, 1.0, 1.0, 0.9], [0.05, 0.0, 0.05, 0.1, 0.05, 0.1, 0.1, 0.1, 0.05, 0.05, 0.05, 0.05, 0.0, 0.2, 0.15, 0.1, 0.2, 0.25, 0.0, 0.25, 0.15, 0.35, 0.3, 0.25, 0.25, 0.3, 0.35, 0.55, 0.3, 0.5, 0.4, 0.45, 0.5, 0.65, 0.4, 0.4, 0.55, 0.5, 0.65, 0.7, 0.7, 0.75, 0.6, 0.85, 0.95, 0.85, 0.75, 0.9, 0.9, 0.9, 0.8, 0.9, 0.85, 0.75, 0.9, 0.9, 0.8, 1.0, 0.95, 0.9, 0.9, 1.0, 0.9, 1.0, 0.9, 1.0, 1.0, 0.95, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0], [0.0, 0.05, 0.1, 0.05, 0.15, 0.0, 0.05, 0.15, 0.1, 0.2, 0.05, 0.15, 0.25, 0.1, 0.2, 0.3, 0.45, 0.25, 0.0, 0.35, 0.4, 0.2, 0.4, 0.2, 0.25, 0.25, 0.7, 0.5, 0.35, 0.4, 0.7, 0.35, 0.45, 0.65, 0.35, 0.8, 0.85, 0.8, 0.55, 0.65, 0.6, 0.85, 0.85, 0.75, 0.75, 0.9, 0.8, 0.9, 0.85, 0.9, 0.9, 1.0, 0.85, 0.8, 0.95, 0.95, 1.0, 0.9, 0.9, 0.9, 0.9, 1.0, 0.9, 0.9, 1.0, 0.95, 1.0, 0.9, 0.95, 0.95, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.05, 0.0, 0.1, 0.15, 0.25, 0.15, 0.0, 0.1, 0.25, 0.05, 0.35, 0.15, 0.2, 0.35, 0.05, 0.3, 0.25, 0.2, 0.25, 0.4, 0.3, 0.35, 0.3, 0.5, 0.35, 0.3, 0.4, 0.7, 0.5, 0.5, 0.5, 0.55, 0.7, 0.6, 0.5, 0.75, 0.8, 0.65, 0.65, 0.85, 0.85, 0.7, 0.7, 0.85, 0.75, 0.85, 0.75, 0.75, 1.0, 0.8, 1.0, 0.8, 0.75, 0.95, 0.9, 0.95, 1.0, 0.95, 1.0, 1.0, 1.0, 0.95, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.0, 0.0, 0.05, 0.05, 0.1, 0.05, 0.1, 0.2, 0.2, 0.05, 0.25, 0.1, 0.25, 0.1, 0.2, 0.05, 0.45, 0.25, 0.55, 0.45, 0.55, 0.4, 0.45, 0.6, 0.35, 0.55, 0.55, 0.55, 0.55, 0.55, 0.65, 0.65, 0.7, 0.65, 0.7, 0.65, 0.5, 0.75, 0.5, 0.8, 0.9, 0.75, 0.75, 0.9, 0.85, 0.95, 0.85, 0.8, 0.85, 0.95, 0.9, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 0.85, 0.9, 1.0, 0.95, 1.0, 1.0, 0.95, 0.9, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.05, 0.0, 0.15, 0.1, 0.05, 0.3, 0.05, 0.15, 0.1, 0.2, 0.2, 0.15, 0.2, 0.4, 0.15, 0.45, 0.3, 0.25, 0.3, 0.45, 0.3, 0.4, 0.55, 0.35, 0.5, 0.35, 0.55, 0.55, 0.6, 0.45, 0.7, 0.6, 0.9, 0.65, 0.45, 0.85, 0.5, 0.7, 0.75, 0.75, 0.95, 0.85, 0.8, 0.8, 0.8, 0.95, 0.9, 0.95, 1.0, 0.85, 0.95, 1.0, 0.95, 0.95, 1.0, 0.85, 0.95, 1.0, 1.0, 0.9, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.1, 0.0, 0.05, 0.1, 0.1, 0.1, 0.15, 0.3, 0.2, 0.2, 0.25, 0.3, 0.25, 0.3, 0.3, 0.45, 0.55, 0.55, 0.4, 0.25, 0.55, 0.6, 0.55, 0.45, 0.6, 0.35, 0.7, 0.6, 0.7, 0.8, 0.65, 0.65, 0.75, 0.55, 0.8, 0.85, 0.75, 0.8, 0.85, 0.8, 0.9, 0.95, 0.9, 1.0, 0.9, 0.8, 0.95, 0.9, 0.85, 0.9, 1.0, 0.95, 1.0, 1.0, 1.0, 0.95, 1.0, 0.95, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.1, 0.05, 0.05, 0.4, 0.15, 0.2, 0.05, 0.2, 0.4, 0.25, 0.15, 0.3, 0.45, 0.4, 0.25, 0.3, 0.5, 0.4, 0.6, 0.45, 0.35, 0.45, 0.6, 0.5, 0.6, 0.65, 0.75, 0.4, 0.55, 0.75, 0.7, 0.85, 0.7, 0.6, 0.9, 0.85, 0.85, 0.8, 0.95, 0.95, 0.8, 0.95, 0.95, 1.0, 1.0, 0.85, 0.95, 0.8, 0.85, 1.0, 0.95, 1.0, 1.0, 0.95, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.1, 0.1, 0.25, 0.15, 0.15, 0.25, 0.2, 0.15, 0.25, 0.4, 0.35, 0.25, 0.25, 0.2, 0.3, 0.5, 0.4, 0.55, 0.3, 0.65, 0.55, 0.7, 0.55, 0.5, 0.7, 0.85, 0.75, 0.75, 0.9, 0.75, 0.65, 0.55, 0.8, 0.9, 0.75, 0.9, 0.8, 0.8, 0.9, 0.85, 0.85, 1.0, 0.95, 0.85, 1.0, 1.0, 1.0, 0.9, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.1, 0.2, 0.1, 0.05, 0.1, 0.1, 0.25, 0.2, 0.4, 0.3, 0.45, 0.35, 0.35, 0.3, 0.5, 0.35, 0.45, 0.55, 0.4, 0.4, 0.4, 0.45, 0.65, 0.7, 0.6, 0.7, 0.65, 0.85, 0.75, 0.8, 0.75, 0.95, 0.8, 0.95, 0.85, 0.8, 0.85, 0.9, 0.85, 0.95, 0.9, 1.0, 0.95, 0.85, 0.9, 1.0, 0.95, 0.95, 1.0, 0.95, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.1, 0.1, 0.0, 0.1, 0.3, 0.2, 0.25, 0.4, 0.5, 0.2, 0.5, 0.6, 0.6, 0.55, 0.4, 0.45, 0.7, 0.5, 0.55, 0.5, 0.75, 0.65, 0.65, 0.65, 0.75, 0.55, 0.6, 0.85, 0.8, 0.8, 0.8, 0.75, 0.85, 0.7, 0.65, 0.8, 0.85, 0.95, 1.0, 0.9, 1.0, 1.0, 0.95, 0.9, 0.95, 1.0, 1.0, 1.0, 0.95, 0.95, 1.0, 0.95, 0.95, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.2, 0.1, 0.35, 0.2, 0.2, 0.35, 0.1, 0.25, 0.15, 0.6, 0.5, 0.5, 0.35, 0.4, 0.6, 0.5, 0.45, 0.6, 0.65, 0.6, 0.75, 0.85, 0.75, 0.85, 0.75, 0.75, 0.75, 0.85, 0.85, 0.9, 0.9, 0.85, 0.95, 0.95, 0.95, 0.95, 0.95, 1.0, 0.9, 1.0, 1.0, 1.0, 0.95, 1.0, 0.95, 1.0, 0.95, 1.0, 0.95, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.1, 0.1, 0.15, 0.2, 0.3, 0.35, 0.6, 0.4, 0.3, 0.4, 0.35, 0.75, 0.4, 0.55, 0.45, 0.75, 0.55, 0.65, 0.6, 0.6, 0.8, 0.85, 0.9, 0.7, 0.6, 0.65, 0.75, 0.85, 0.8, 0.85, 0.9, 0.9, 0.75, 0.9, 1.0, 1.0, 0.95, 0.95, 1.0, 0.95, 0.95, 0.95, 0.95, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.05, 0.05, 0.1, 0.15, 0.2, 0.4, 0.4, 0.45, 0.5, 0.45, 0.55, 0.65, 0.65, 0.65, 0.7, 0.55, 0.45, 0.75, 0.65, 0.65, 0.85, 0.8, 0.8, 0.65, 0.9, 0.8, 0.9, 0.9, 0.65, 0.9, 0.9, 0.85, 1.0, 0.9, 0.95, 0.85, 0.95, 0.95, 0.95, 1.0, 0.95, 0.95, 1.0, 0.95, 0.95, 0.95, 1.0, 0.95, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.15, 0.25, 0.1, 0.3, 0.15, 0.4, 0.3, 0.6, 0.45, 0.45, 0.5, 0.5, 0.55, 0.7, 0.75, 0.5, 0.65, 0.8, 0.7, 0.75, 0.85, 0.9, 0.75, 0.8, 0.8, 0.7, 0.9, 1.0, 1.0, 0.85, 0.95, 0.9, 1.0, 1.0, 1.0, 0.95, 0.95, 1.0, 0.95, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.45, 0.2, 0.1, 0.35, 0.4, 0.45, 0.3, 0.35, 0.6, 0.45, 0.45, 0.65, 0.55, 0.8, 0.85, 0.65, 0.75, 0.75, 0.65, 0.95, 0.7, 0.8, 0.9, 0.75, 0.9, 0.9, 0.8, 0.85, 0.9, 1.0, 0.9, 0.95, 0.95, 0.9, 0.95, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.25, 0.1, 0.15, 0.35, 0.35, 0.4, 0.8, 0.55, 0.6, 0.65, 0.55, 0.7, 0.35, 0.65, 0.75, 0.75, 0.85, 0.7, 0.65, 0.9, 0.9, 0.85, 0.85, 0.9, 0.95, 0.95, 0.95, 0.95, 0.85, 0.9, 1.0, 0.9, 0.85, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.3, 0.35, 0.4, 0.25, 0.2, 0.35, 0.75, 0.7, 0.3, 0.6, 0.55, 0.75, 0.85, 0.6, 0.85, 0.75, 0.45, 0.85, 0.75, 0.9, 1.0, 1.0, 0.8, 0.9, 1.0, 0.9, 0.85, 0.95, 0.85, 1.0, 0.9, 1.0, 0.9, 1.0, 1.0, 0.95, 0.95, 1.0, 1.0, 0.9, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.2, 0.35, 0.45, 0.4, 0.5, 0.4, 0.4, 0.6, 0.7, 0.8, 0.85, 0.9, 0.6, 0.45, 0.85, 0.7, 0.8, 0.85, 0.8, 0.8, 0.85, 0.8, 0.9, 0.9, 0.95, 1.0, 0.9, 1.0, 0.95, 1.0, 0.95, 1.0, 0.95, 1.0, 0.9, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.15, 0.35, 0.45, 0.4, 0.3, 0.6, 0.6, 0.65, 0.7, 0.6, 0.85, 0.75, 0.75, 0.85, 0.75, 0.95, 0.9, 0.7, 0.8, 0.85, 0.9, 0.9, 0.95, 0.85, 1.0, 0.9, 0.85, 0.85, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.3, 0.15, 0.6, 0.6, 0.5, 0.65, 0.6, 0.7, 0.5, 0.65, 0.75, 0.9, 0.8, 0.8, 0.75, 0.95, 0.85, 1.0, 0.9, 0.85, 0.9, 1.0, 0.9, 0.95, 1.0, 0.85, 0.95, 0.85, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.95, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.4, 0.25, 0.3, 0.35, 0.6, 0.55, 0.55, 0.6, 0.65, 0.75, 0.85, 0.95, 0.9, 1.0, 0.9, 0.95, 0.95, 0.9, 0.8, 0.9, 0.85, 1.0, 0.85, 0.95, 0.9, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 0.95, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.4, 0.3, 0.4, 0.65, 0.55, 0.4, 0.6, 0.75, 0.8, 0.65, 0.6, 0.9, 0.9, 0.9, 0.8, 0.8, 0.9, 1.0, 0.9, 0.95, 0.9, 0.95, 1.0, 0.95, 0.95, 1.0, 0.95, 0.95, 1.0, 1.0, 0.95, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.45, 0.35, 0.6, 0.55, 0.4, 0.75, 0.8, 0.65, 0.9, 0.75, 0.75, 0.8, 0.9, 0.9, 1.0, 0.9, 0.85, 0.85, 0.9, 1.0, 0.9, 1.0, 0.9, 1.0, 1.0, 0.95, 0.9, 1.0, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.35, 0.55, 0.65, 0.7, 0.65, 0.7, 0.65, 0.75, 0.85, 0.95, 0.85, 0.95, 0.8, 0.8, 0.95, 0.85, 1.0, 0.9, 1.0, 0.85, 0.9, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.5, 0.5, 0.65, 0.75, 0.6, 0.75, 0.85, 0.85, 0.8, 0.9, 0.85, 0.85, 0.85, 1.0, 1.0, 0.7, 0.95, 0.95, 1.0, 0.9, 0.95, 0.95, 0.95, 1.0, 0.85, 0.9, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.3, 0.45, 0.65, 0.55, 0.6, 0.7, 0.7, 0.8, 0.95, 0.9, 0.75, 1.0, 0.9, 0.95, 0.9, 0.95, 1.0, 1.0, 0.9, 0.95, 1.0, 0.95, 1.0, 0.95, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.4, 0.5, 0.6, 0.55, 0.55, 0.75, 0.95, 0.7, 0.75, 0.85, 0.9, 0.9, 0.85, 0.85, 1.0, 0.9, 0.85, 1.0, 1.0, 1.0, 1.0, 0.9, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.6, 0.6, 0.7, 0.6, 0.6, 0.75, 0.7, 0.75, 0.85, 0.9, 0.95, 0.95, 0.95, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 0.95, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.6, 0.6, 0.75, 0.55, 0.75, 0.8, 1.0, 0.9, 0.95, 0.9, 0.95, 0.9, 0.9, 1.0, 0.9, 0.95, 0.95, 0.95, 0.95, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 0.95, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.6, 0.7, 0.65, 0.75, 0.95, 1.0, 0.8, 0.95, 0.85, 1.0, 0.85, 0.95, 0.85, 0.95, 0.95, 1.0, 0.95, 1.0, 0.95, 1.0, 0.95, 1.0, 0.95, 1.0, 1.0, 0.95, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.55, 0.75, 0.85, 0.9, 0.8, 0.95, 0.8, 0.85, 1.0, 0.9, 0.95, 0.95, 0.9, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.45, 0.6, 0.8, 0.85, 0.95, 0.9, 0.9, 1.0, 1.0, 0.9, 0.9, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.65, 0.85, 0.85, 0.95, 0.95, 0.8, 0.95, 0.9, 0.95, 1.0, 1.0, 0.95, 1.0, 1.0, 0.95, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.7, 0.8, 0.9, 0.8, 0.9, 0.9, 0.95, 0.95, 0.95, 1.0, 0.95, 1.0, 1.0, 1.0, 0.9, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.85, 0.9, 0.7, 0.9, 0.8, 0.95, 0.95, 1.0, 0.95, 1.0, 1.0, 0.95, 0.95, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.85, 0.95, 0.9, 0.95, 0.95, 0.9, 0.9, 0.95, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 0.9, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.7, 0.75, 0.9, 1.0, 0.95, 0.85, 0.95, 0.9, 1.0, 0.9, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.7, 0.9, 0.9, 0.9, 0.95, 0.95, 1.0, 0.9, 1.0, 0.95, 1.0, 1.0, 1.0, 0.9, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.9, 1.0, 0.85, 0.9, 1.0, 1.0, 0.9, 0.95, 0.95, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [1.0, 0.85, 0.9, 0.9, 0.95, 0.95, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.85, 0.9, 0.85, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.95, 0.9, 0.95, 0.95, 0.95, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 0.95, 0.95, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.95, 1.0, 0.9, 0.95, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.95, 1.0, 0.9, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]]

In [29]:
import collections, numpy

flat_stats1 = [item for sublist in stats1 for item in sublist]
print(collections.Counter(flat_stats1))
print(np.count_nonzero(flat_stats1))
print(len(flat_stats1))

flat_stats2 = [item for sublist in stats2 for item in sublist]
print(collections.Counter(flat_stats2))
print(np.count_nonzero(flat_stats2))

Counter({1.0: 3311, 0.0: 2388, 0.05: 346, 0.95: 332, 0.1: 198, 0.9: 178, 0.15: 159, 0.8: 140, 0.2: 138, 0.85: 135, 0.7: 113, 0.3: 110, 0.5: 108, 0.25: 107, 0.75: 105, 0.65: 104, 0.6: 94, 0.35: 91, 0.55: 82, 0.45: 81, 0.4: 80})
6012
8400
Counter({1.0: 3979, 0.0: 1978, 0.05: 331, 0.95: 299, 0.9: 216, 0.1: 170, 0.85: 146, 0.15: 129, 0.8: 107, 0.25: 105, 0.75: 104, 0.2: 103, 0.6: 88, 0.4: 87, 0.7: 86, 0.55: 85, 0.65: 83, 0.35: 82, 0.3: 82, 0.45: 74, 0.5: 66})
6422
